# "Spam Filter" Algorithm with Naive Bayes

Our objective is to develop an algorithm to detect sapm SMS messages.
To classify messages as spam or non-spam, the algortihm will:

- Learns how humans classify messages.
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam 


To do that, we'll use the "multinomial Naive Bayes" algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.
 
The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The dataset can be downloaded directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on this [page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where there are also some of the authors' papers.




--------
Exploring Test Set
-----------

In [1]:
import pandas as pd
ssc = pd.read_csv('SMSSpamCollection', sep='\t', header=None,
                  names=['Label', 'SMS'])

In [2]:
ssc.sample(20)

,Label,SMS
165,spam,BangBabes Ur order is on the way. U SHOULD rec...
4032,ham,"Sorry vikky, i'm Watching olave mandara movie ..."
5019,ham,Babe ! What are you doing ? Where are you ? Wh...
4928,ham,Wanna do some art?! :D
4480,ham,Erutupalam thandiyachu
1801,ham,excellent. I spent &lt;#&gt; years in the Ai...
597,ham,Gud mrng dear have a nice day
4531,ham,Don't forget though that I love you .... And I...
1706,ham,Yun ah.now ü wkg where?btw if ü go nus sc. Ü w...
2017,ham,"Princess, is your kitty shaved or natural?"


In [3]:
# number of rows and columns in dataset
ssc.shape

(5572, 2)

In [4]:
# percentage of spam and non-spam (ham)
round(ssc['Label'].value_counts(normalize=True) * 100, 1)

ham     86.6
spam    13.4
Name: Label, dtype: float64

------
Training and Test Set
---------

To test how good the algorithm will be with classifying new messages we'll need to test. To test the spam filter, we're first going to split our dataset into two categories:

- A training set, which we'll use to "train" the computer how to classify messages.
- A test set, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing.

Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%.

In [5]:
# randomizing the entire dataset
ssc_rand = ssc.sample(frac=1, random_state=1)

In [6]:
# spliting 80% - 20%
row_count = ssc.shape[0]
eighty_pct = int(0.8 * row_count) + 1

# training dataset & reseting index labels
ssc_tr = ssc_rand[:eighty_pct].reset_index(drop=True)

# testing dataset & reseting index labels
ssc_te = ssc_rand[eighty_pct:] .reset_index(drop=True)

print(len(ssc_tr))
print(len(ssc_te))

4458
1114


In [7]:
# percentage of spam and ham in training datset
round(ssc_tr['Label'].value_counts(normalize=True) * 100, 1)

ham     86.5
spam    13.5
Name: Label, dtype: float64

In [8]:
# percentage of spam and ham in testing datset
round(ssc_te['Label'].value_counts(normalize=True) * 100, 1)

ham     86.8
spam    13.2
Name: Label, dtype: float64

The percentage of spam and ham in  the training and the test set are similar

-------------
Data Cleaning
------------

#### Letter Case and Punctuaton
removing the punctuation and changed all letters to lowercase.

In [9]:
ssc_tr.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
# training dataset
# removing punctuations and transforming every letter in every word to lower case
ssc_tr['SMS'] = ssc_tr['SMS'].str.replace('\W', ' ').str.lower()


In [11]:
# after cleaning
ssc_tr.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


#### Creating the Set of Unique Words (Vocabulary)

In [12]:
# transforming each message from the SMS column into a list by splitting the string at the space character
ssc_tr['SMS'] = ssc_tr['SMS'].str.split()

In [13]:
ssc_tr.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


In [14]:
vocabulary = []
for row in ssc_tr['SMS']:
    for i in row:
        vocabulary.append(i)
        
vocabulary = list(set(vocabulary))

In [15]:
vocabulary[:10]

['walls',
 'professional',
 'allday',
 'eyes',
 'caught',
 'millions',
 'deeraj',
 '09061790125',
 'wherre',
 'regarding']

In [16]:
len(vocabulary)  

7783

#### Final Training Set

In [17]:
# counting number of words from "Vocabulary" in each SMS
dict_count = {i: [0] * len(ssc_tr['SMS']) for i in vocabulary}

for index, row in enumerate(ssc_tr['SMS']):
    for j in row:
        dict_count[j][index] += 1

In [18]:
df_count = pd.DataFrame(dict_count)

In [19]:
df_count.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [20]:
# adding the label column to get final training set
fts = pd.concat([ssc_tr, df_count], axis='columns')

In [21]:
fts.sample(5)

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
3570,spam,"[free, ringtone, reply, real, or, poly, eg, re...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1792,spam,"[want, to, funk, up, ur, fone, with, a, weekly...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3373,ham,"[ta, daaaaa, i, am, home, babe, are, you, stil...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4294,ham,"[reverse, is, cheating, that, is, not, mathema...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4046,ham,"[sorry, it, s, a, lot, of, friend, of, a, frie...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


-------------
Calculating Constants & Parametrs for Naive Bayes Algorithm
--------------

See [link](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) for the details of Naive Bayes algorithm.

First, we need to calculate the following constants:
 - P(Spam): probability of spam
 - P(Ham): probability of ham
 - N<sub>Spam</sub>: number of all words in spam (not necessarily unique)
 - N<sub>Ham</sub>: number of all words in ham (not necessarily unique)
 - N<sub>Vocabulary</sub>: number of all unique words in the training set
 
 We'll also use Laplace smoothing and set α = 1


In [22]:
# calculating P(Spam) & P(Ham)
fts_spam = fts[fts['Label']=='spam']
fts_ham = fts[fts['Label']=='ham']


p_spam = len(fts_spam) / len(fts)
p_ham = len(fts_ham) / len(fts)

In [23]:
# calculating N_spam, N_ham * N_vocaulary
n_spam = sum([len(i) for i in fts_spam['SMS']])
n_ham = sum([len(i) for i in fts_ham['SMS']])
n_voc = len(vocabulary)

In [24]:
# Laplace smoothing factor
alpha = 1

Now we need to calculate teh follwoing parameteres from the training set:
- P(w<sub>i</sub>|Spam): probability of a word (w<sub>i</sub>) given a spam message
- P(w<sub>i</sub>|Ham): probability of a word (w<sub>i</sub>) given a ham message

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [25]:
p_wi_spam = {i:0 for i in vocabulary}
p_wi_ham = {i:0 for i in vocabulary}

# calculating P(wi|Spam) & P(wi|Ham) base on formulas above
for i in vocabulary:
    n_wi_spam = fts_spam[i].sum()
    n_wi_ham = fts_ham[i].sum()
    
    p_wi_spam[i] = (n_wi_spam + alpha) / (n_spam + alpha * n_voc)
    p_wi_ham[i] = (n_wi_ham + alpha) / (n_ham + alpha * n_voc)


------------
Classifying a New Message
------------

The spam filter should take in as input a new message (w1, w2, ..., wn)
Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
- If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
- If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
- If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

Based on Naive Bayes algorithm:
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

In [26]:
import re

def classify(message):

    # message is assumed to be a string
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # initiating P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
    p_spam_message = p_spam
    p_ham_message = p_ham
    
    for wi in message:
        if wi in p_wi_spam:
            p_spam_message *= p_wi_spam[wi]
        if wi in p_wi_ham:
            p_ham_message *= p_wi_ham[wi]

    print('P(Spam|message) / P(Ham|message):', 
          p_spam_message / p_ham_message)

    if p_ham_message > p_spam_message:
        print('Label: Ham')
    elif p_ham_message < p_spam_message:
        print('Label: Spam')
    else:
        print('Equal proabilities')

In [27]:
# testing the function
message1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
message2 = "Sounds good, Tom, then see u there"

classify(message1)
classify(message2)


P(Spam|message) / P(Ham|message): 69.60582447618044
Label: Spam
P(Spam|message) / P(Ham|message): 6.609403256580054e-05
Label: Ham


--------------
Measuring Spam Algorithm's Accuracy
-------------

We'll now try to determine how well the algorithm performs on our test set of 1,114 messages

In [32]:
def classify_testds(message):

    # message is assumed to be a string
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # initiating P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
    p_spam_message = p_spam
    p_ham_message = p_ham
    
    for wi in message:
        if wi in p_wi_spam:
            p_spam_message *= p_wi_spam[wi]
        if wi in p_wi_ham:
            p_ham_message *= p_wi_ham[wi]
  
    if p_ham_message > p_spam_message:
        return 'ham'
    elif p_ham_message < p_spam_message:
        return 'spam'
    else:
        return 'Equal proabilities'

In [34]:
ssc_te['Prediction'] = ssc_te['SMS'].apply(classify_testds)
ssc_te.sample(10)

,Label,SMS,Prediction
240,ham,.Please charge my mobile when you get up in mo...,ham
438,spam,You have an important customer service announc...,spam
299,ham,I not at home now lei...,ham
1057,ham,Nope. I just forgot. Will show next week,ham
625,spam,URGENT! We are trying to contact U. Todays dra...,spam
901,spam,Double mins and txts 4 6months FREE Bluetooth ...,spam
73,ham,"Chinatown got porridge, claypot rice, yam cake...",ham
416,ham,I wonder if you'll get this text?,ham
904,ham,"Greetings me, ! Consider yourself excused.",ham
1052,ham,Babe! How goes that day ? What are you up to ?...,ham


Measuring the accuracy of the  algorthm:

In [48]:
total = len(ssc_te)
correct = len(ssc_te[ssc_te['Label']==ssc_te['Prediction']])

accuracy = round(correct / total * 100, 1)

print(accuracy)


98.7


Accuracy of the algorithm is close to 99% which fulfills our initial objective (>80%)!

finding the cases where the algorithm failed:
    

In [52]:
not_correct =  ssc_te[ssc_te['Label']!=ssc_te['Prediction']]
not_correct

,Label,SMS,Prediction
114,spam,Not heard from U4 a while. Call me now am here...,ham
135,spam,More people are dogging in your area now. Call...,ham
152,ham,Unlimited texts. Limited minutes.,spam
159,ham,26th OF JULY,spam
284,ham,Nokia phone is lovly..,spam
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,Equal proabilities
302,ham,No calls..messages..missed calls,spam
319,ham,We have sent JD for Customer Service cum Accou...,spam
504,spam,Oh my god! I've found your number again! I'm s...,ham
546,spam,"Hi babe its Chloe, how r u? I was smashed on s...",ham


Out of 14 cases with wrong prediction, one has "equal probability", 8 are predicted to be ham by mistake and 5 are predicted to be spam by mistake (so wrong detection on both sides).

som observations (on this limited number of wrong predictions):
- 4 out of 5 messages mistaken for spam are very short
- many of messages mistaken for ham include one or more abbreviations, numbers or special characters